In [1]:
import pandas as pd
import numpy as np
import os
from typing import Callable # this is to define test_func functions dtype as function itself
import re # for regural expression
import warnings
warnings.filterwarnings('ignore')
import pickle # for pickling my model

# Can import the below dataset from 
/kaggle/input/mobile-recommendation-system-dataset/mobile_recommendation_system_dataset.csv

In [2]:
base_path = os.getcwd()
sub_path = 'data'
dataset_path = os.path.join(base_path, sub_path, 'mobile_recommendation_system_dataset.csv')

In [3]:
df = pd.read_csv(dataset_path,index_col=False)

In [4]:
df.head()

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...


In [5]:
def random_values(col_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [6]:
random_values('corpus')

2472 : Storage64 GBRAM4 GBExpandable Storage256GB  SystemAndroid 12Processor TypeMediaTek G37Processor Speed2.3 13MP 8MP 8MP 4G Capacity6000 Display Size17.32 cm (6.82 inch)Resolution720 X 1640 PixelsResolution TypeHD+Display TypeHD+ IPSOther Display FeaturesPunch-Hole Display, 90Hz Refresh Rate, 120Hz Touch Sampling Rate, Brightness Upto 450 NITS, 99.8% sRGB Color Gamut, 1500:1 Color Contrast Ratio, 21:9 Aspect Ratio, Eye Care Mode
1955 : Storage128 GBRAM8  SystemAndroid Android 13Processor Speed2.4 50MP 5G Capacity6000 Display Size16.51 cm (6.5 inch)Resolution2340X1080
1098 : Storage128 GBRAM8  SystemAndroid OxygenOS based on Android 13Processor Speed3.2 50MP 5G Capacity5000 Display Size17.02 cm (6.7 inch)Resolution2772x1240
221 : Storage128 GBRAM8 GBExpandable Storage1TB  SystemAndroid 11Processor TypeQualcomm Snapdragon 750GProcessor Speed2.2 48MP 5MP 5MP 20MP 5G Capacity5000 Display Size16.76 cm (6.6 inch)Resolution1600 x 720 PixelsResolution TypeHD+Display TypeHD+ sAMOLED Display

## Lowercasing the corpus values

In [7]:
df['corpus'] = df['corpus'].str.lower()

In [8]:
random_values('corpus')

2369 : storage128 gbram8 gbexpandable storage1tb  systemandroid 11processor typequalcomm snapdragon 778gprocessor speed2.4 64mp 8mp 5mp 20mp 5g capacity4250 display size16.64 cm (6.55 inch)resolution1080 x 2400resolution typefull hd+gpuadreno 642ldisplay typeamoledother display featuresdolby vision®, 60/90hz refresh rate, hdr10+, corning® gorilla® glass 5
2276 : storage64 gbram3 gbexpandable storage512gb  systemandroid pie 9processor typequalcomm snapdragon 439processor speed1.95 13mp 2mp 13mp 2mp 8mp 8mp 4g capacity5000 display size15.8 cm (6.22 inch)resolution1520 x 720 pixelsresolution typehd+gpuadreno 505 (at 650 mhz)display typehd+ ips displayother display features2.5d glass with corning gorilla glass 5 screen protection, 1500:1 contrast ratio, 70.8% ntsc, supports sunlight mode, supports night mode, supports reading mode, supports color temperature adjustment, support standard mode
534 : storage128 gbram6 gbexpandable storage1tb  systemandroid 13processor typesnapdragon 685proces

In [9]:
df.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2546 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2546 non-null   object 
 1   ratings  2546 non-null   float64
 2   price    2546 non-null   object 
 3   imgURL   2546 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 99.6+ KB


In [11]:
df.dropna(inplace=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2534 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2534 non-null   object 
 1   ratings  2534 non-null   float64
 2   price    2534 non-null   object 
 3   imgURL   2534 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 118.8+ KB


In [13]:
max_len = 0
for i in range(0, len(df['corpus'])):
    try:
        if len(df['corpus'][i]) > max_len:
            max_len = len(df['corpus'][i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max len in corpus is: 1196


In [14]:
df.sample(10)

,name,ratings,price,imgURL,corpus
418,"SAMSUNG Galaxy F41 (Fusion Blue, 128 GB)",4.4,14499,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage512gb s...
1521,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13945,https://rukminim2.flixcart.com/image/312/312/x...,storage64 gbram6 systemandroid 11processor sp...
1465,"REDMI Note 12 Pro 5G (Glacier Blue, 128 GB)",4.2,24999,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram8 systemandroid 12processor t...
1663,"Redmi Note 9 Pro (Glacier White, 128 GB)",4.3,17999,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage512gb s...
363,SAMSUNG Guru FM Plus,4.1,1500,https://rukminim2.flixcart.com/image/312/312/m...,storage128 gbram8 gbexpandable storage1tb fre...
2188,"SAMSUNG Galaxy S23 Ultra 5G (Phantom Black, 25...",4.4,6073,https://rukminim2.flixcart.com/image/312/312/x...,storage256 gbram12 systemandroid 13processor ...
367,"SAMSUNG Galaxy Z Flip3 5G (Phantom Black, 256 GB)",4.3,23463,https://rukminim2.flixcart.com/image/312/312/k...,storage256 gbram8 systemandroid 11processor t...
540,"REDMI K50i 5G (Stealth Black, 128 GB)",4.2,20999,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram6 systemandroid 12processor t...
455,"OPPO A71 (Gold, 16 GB)",4.6,"₹13,490",https://rukminim2.flixcart.com/image/312/312/j...,storage16 gbram3 gbexpandable storage256gb sy...
613,"SAMSUNG Galaxy M01 (Red, 32 GB)",4.3,8999,https://rukminim2.flixcart.com/image/312/312/k...,storage32 gbram3 gbexpandable storage512gb sy...


In [15]:
random_values('imgURL')

574 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/c/z/h/-original-imaghm9y8tceqxmx.jpeg?q=70
2508 : https://rukminim2.flixcart.com/image/312/312/jog2nbk0/mobile/d/j/d/mi-redmi-e7t-na-original-imafazxhztrw9gnr.jpeg?q=70
784 : https://rukminim2.flixcart.com/image/312/312/kr3tj0w0/mobile/m/r/7/y72-5g-v2060-vivo-original-imag4yngeb9dz4fe.jpeg?q=70
512 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/0/h/k/-original-imagcg22czc3ggvw.jpeg?q=70
392 : https://rukminim2.flixcart.com/image/312/312/kqfj1jk0/mobile/h/l/d/hot-10s-x689-64-4-infinix-original-imag4fk5ajmdtwz4.jpeg?q=70
1402 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/s/z/1/spark-9-kg5p-tecno-original-imagghbwdhygjag2.jpeg?q=70


# Model

## vectorization(to convert the words to vectors)   (BOW)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer 

In [17]:
cont_vect = CountVectorizer(max_features=1000)

In [18]:
arrays = cont_vect.fit_transform(df['corpus'])

In [19]:
vectors = arrays.toarray()

In [20]:
cont_vect.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '07', '08', '09', '0mp',
       '0primary', '0processor', '10', '100', '1000', '100000', '1000000',
       '100mp', '103', '104', '106', '1080', '1080processor',
       '1080resolution', '108mp', '10mp', '10processor', '11', '1100',
       '1125', '1170', '1179', '11primary', '11processor', '12', '120',
       '1200', '1200processor', '120hz', '1242', '1280processor', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300', '1330',
       '1334', '135', '13mp', '13processor', '14', '1400', '144', '1440',
       '144hz', '1480', '14processor', '15', '1500', '1544',
       '15processor', '16', '160', '1600', '16000', '1600resolution',
       '1612', '16mp', '16processor', '18', '180', '1800', '180hz', '19',
       '1920', '1920hz', '1processor', '20', '200', '2000000', '200mp',
       '20mp', '21', '22', '23', '2340', '24', '240', '2400',
       '2400resolution', '2408', '240hz', '2460', '24mp', '25mm', '25mp',
       '26', '27'

## Calculating cosine similarity for recommendation

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
similarity = cosine_similarity(vectors)
similarity.shape # this is because I have calculated the similarity among all the rows to all the rows(confusing?? but try to understand)

(2534, 2534)

In [25]:
similarity[2]

array([0.35556531, 0.1548516 , 1.        , ..., 0.58823529, 0.19988898,
       0.42452948])

In [26]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(68, np.float64(0.9999999999999999)),
 (303, np.float64(0.9999999999999999)),
 (616, np.float64(0.9850746268656715)),
 (1461, np.float64(0.9850746268656715)),
 (457, np.float64(0.9701492537313432)),
 (1027, np.float64(0.9701492537313432)),
 (1735, np.float64(0.9701492537313432)),
 (569, np.float64(0.9468131937613656)),
 (1123, np.float64(0.9468131937613656)),
 (1540, np.float64(0.9468131937613656))]

In [27]:
def recommend(mobile):
    mobile_index = df[df['name']==mobile].index[0]
    similarity_array = similarity[mobile_index]
    similar_10_mobiles = sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in similar_10_mobiles:
        print(df['name'].iloc[i[0]])



In [35]:
recommend('APPLE iPhone 13 (Blue, 128 GB)')

vivo Y56 5G (Black Engine, 128 GB)
vivo Y56 5G (Black Engine, 128 GB)
vivo Y56 5G (Orange Shimmer, 128 GB)
vivo T2x 5G (Aurora Gold, 128 GB)
vivo T2x 5G (Glimmer Black, 128 GB)
vivo T2x 5G (Marine Blue, 128 GB)
vivo Y36 (Meteor Black, 128 GB)
vivo Y36 (Vibrant Gold, 128 GB)
vivo T2x 5G (Aurora Gold, 128 GB)
vivo T2x 5G (Marine Blue, 128 GB)


## making Pickle file of df and similarities to use in web

In [23]:
os.makedirs(r'src\model',exist_ok=True)

In [24]:
pickle.dump(obj=df,file=open(file=r'src\model\dataframe.pkl',mode='wb'))

In [25]:
pickle.dump(obj=similarity,file=open(file=r'src\model\similarity.pkl',mode='wb'))